## Chapter 2-5, 2강 TTS(텍스트→음성) 기초 — Google Cloud TTS

- 목표: TTS 기본 파이프라인 이해, GCP 보이스/언어 활용, SSML 억양 제어


### 학습 목표
- 텍스트를 자연스러운 음성으로 합성하는 전체 흐름 이해
- GCP Text-to-Speech API로 보이스 선택, SSML 제어, 멀티언어 생성/번역 실습
- 품질·비용·속도 트레이드오프를 체감하고 목적별 설정 가이드 수립

### 커리큘럼 (Overview)
0) ** 환경 준비 및 라이브러리 임포트 **

1) **Google Cloud TTS — 설치·설정 & 최소 동작(Quickstart)**

3) **텍스트 정규화/문장 분할 실습(소수점 보호)**  

4) **보이스/언어 탐색(멀티스피커.멀티언어 확인)**  

5) **보이스 선택 가이드(GCP)**  

6) **SSML 미세조정 체크리스트 (A/B)**  





### 0. 환경 준비 및 라이브러리 임포트


In [ ]:
# 플랫폼별 참고 (libsndfile 관련)
# - soundfile(pysoundfile)은 시스템의 'libsndfile' 라이브러리에 의존합니다.
# - 보통 pip로 설치한 soundfile 휠이 OS별로 libsndfile을 함께 포함하지만,
#   환경에 따라 ImportError/OSError가 발생할 수 있어 아래 대응을 권장합니다.

# macOS: brew install libsndfile
# Windows: poetry add soundfile (pip install soundfile)

#### GCP의 Credential 및 API() Enable
![GCP_Credentials_0](./images/gcp_credentials_0.png)
![GCP_Credentials_1](./images/gcp_credentials_1.png)
![GCP_Credentials_2](./images/gcp_credentials_2.png)
![GCP_Credentials_3](./images/gcp_credentials_3.png)
![GCP_Credentials_4](./images/gcp_credentials_4.png)
![GCP_API_5](./images/gcp_api_5.png)
![GCP_API_6](./images/gcp_api_6.png)
![GCP_API_7](./images/gcp_api_7.png)
![GCP_API_8](./images/gcp_api_8.png)
![GCP_API_9](./images/gcp_api_9.png)

In [ ]:
# 목적:
# - 경고 메시지 억제(실습/데모 시 화면 노이즈 감소)
# - Matplotlib 한글 폰트 설정 및 음수 기호 깨짐 방지
# - PyTorch 계산 장치(GPU/CPU) 자동 선택 출력

import os, warnings
import numpy as np
import matplotlib.pyplot as plt
import torch, torchaudio

from google.cloud import texttospeech

# 정규화/문장 분할 실습 시 필요
import re

# ─────────────────────────────────────────────────────────
# 1) 경고 억제: 실습/노트북에서 필요 없는 경고를 숨겨 가독성 향상
#    (주의) 디버깅이 필요할 땐 이 섹션을 잠시 주석 처리하세요.
# ─────────────────────────────────────────────────────────
warnings.filterwarnings("ignore", category=FutureWarning)        # 향후 변경 예고(FutureWarning) 숨김
warnings.filterwarnings("ignore", category=DeprecationWarning)   # 사용 중단 예정 API 경고 숨김
warnings.filterwarnings("ignore", message=r"Glyph.*missing from font.*", category=UserWarning)  # 폰트 글리프 경고 숨김
warnings.filterwarnings("ignore", message=".*load_with_torchcodec.*", category=UserWarning)     # torchaudio codec 관련 경고 숨김
warnings.filterwarnings("ignore", message=".*save_with_torchcodec.*", category=UserWarning)     # torchaudio codec 관련 경고 숨김
warnings.filterwarnings("ignore")  # 최후의 보루(모든 경고 숨김) — 필요 시만 사용

# ─────────────────────────────────────────────────────────
# 2) Matplotlib 한글/기호 렌더링 설정
#    - macOS: 'AppleGothic'
#    - Windows: 'Malgun Gothic'
#    - Linux: 'NanumGothic' 등을 권장
# ─────────────────────────────────────────────────────────
plt.rcParams['font.family'] = 'AppleGothic'  # 한글 깨짐 방지(환경에 맞는 폰트로 교체 가능)
plt.rcParams['axes.unicode_minus'] = False   # 한글 폰트 사용 시 음수 기호(−) 깨짐 방지

# ─────────────────────────────────────────────────────────
# 3) 계산 장치 선택: GPU 가용 시 'cuda', 아니면 'cpu'
# ─────────────────────────────────────────────────────────
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
print('Device:', DEVICE)

# (안전) 출력 폴더 보장
os.makedirs("./data/speech", exist_ok=True)


### 1. Google Cloud TTS — 설치·설정 & 최소 동작(Quickstart)
- ensure()로 필수 패키지 보장 후 .env를 로드해 자격 설정, TTS 클라이언트를 생성합니다.
- 최소 합성 함수로 텍스트를 WAV(16-bit PCM, 24kHz)로 저장하고, 스모크 테스트로 bench_gcp.wav를 생성해 연결을 확인합니다.

In [ ]:
# 목적:
# - 필요한 패키지가 없으면 자동 설치해 주는 간단한 헬퍼를 정의하고,
#   본 노트북에서 사용하는 의존성(google-cloud-texttospeech, soundfile)을 보장합니다.

import importlib, sys, subprocess  # 동적 import, 현재 파이썬 실행파일 경로, pip 호출용

def ensure(pkg: str, pip_name: str | None = None):
    """
    pkg: import 경로 (예: 'google.cloud.texttospeech', 'soundfile')
    pip_name: pip 패키지명 (예: 'google-cloud-texttospeech'); 생략 시 pkg 사용
      - import 경로와 pip 패키지명이 다른 경우에만 지정합니다.
      - 버전 고정이 필요하면 '패키지==버전' 형태로 넘기세요 (예: 'torchaudio==2.2.0').

    동작:
    1) importlib.import_module(pkg) 시도로 설치 유무 확인
    2) 실패 시, 현재 커널/가상환경에 맞춰 `python -m pip install` 실행

    주의:
    - 사내 네트워크/프록시 환경에선 pip 접속이 제한될 수 있습니다.
    - 일부 패키지는 설치 후 커널 재시작이 필요할 수 있습니다.
    """
    name = pip_name or pkg
    try:
        importlib.import_module(pkg)  # 이미 설치되어 있으면 여기서 통과
    except Exception:
        print(f'Installing {name} ...')
        # sys.executable을 사용해 현재 커널(venv/conda)에 설치되도록 보장
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', name, '--quiet'])



# google-cloud-texttospeech:
# - import 경로: 'google.cloud.texttospeech'  ⟷  pip 패키지명: 'google-cloud-texttospeech' (이름이 다름)
ensure('google.cloud.texttospeech', 'google-cloud-texttospeech')

# soundfile:
# - import 경로와 pip 패키지명이 동일
ensure('soundfile')



In [ ]:
# 목적:
# - .env 파일에 저장된 환경변수(예: GOOGLE_APPLICATION_CREDENTIALS, GOOGLE_CLOUD_PROJECT 등)를
#   프로세스 환경으로 불러와 GCP 인증·설정 값을 코드에서 편리하게 사용하기 위함.
# - service_account 모듈은 이후 GCP 클라이언트 생성 시 서비스 계정 키(JSON)로
#   Credentials 객체를 만들 때 사용.

import os                           # 환경변수 접근(os.getenv) 및 경로 검사(os.path.exists)용
from dotenv import load_dotenv      # .env 파일을 읽어 환경변수로 로드
from google.oauth2 import service_account  # 서비스 계정 기반 Credentials 생성을 위한 모듈
from google.cloud import translate_v2 as translate

# .env 파일 로드:
# - 기본적으로 현재 작업 디렉터리의 ".env" 파일을 읽어들여 os.environ에 주입합니다.
# - 반환값은 True/False 이며, 파일이 없더라도 에러를 내지 않고 그냥 False를 반환합니다.
# - 보안 주의: .env와 서비스 계정 JSON 키는 절대 Git에 커밋하지 말고 .gitignore에 추가하세요.
load_dotenv()


In [ ]:
# 목적:
# - GCP Text-to-Speech 클라이언트를 생성하고
# - 주어진 텍스트를 16-bit PCM(WAV) 파일로 저장하는 기본 흐름을 보여줍니다.

from google.cloud import texttospeech
import wave
import os
from google.oauth2 import service_account

def make_tts_client():
    """
    GCP TTS 클라이언트 생성 유틸.
    우선순위:
      1) GOOGLE_APPLICATION_CREDENTIALS 환경변수로 지정된 서비스 계정 키(JSON) 사용
      2) 없으면 ADC(Application Default Credentials)에 폴백
         - 예: gcloud auth application-default login 으로 설정된 자격 등
    반환:
      - texttospeech.TextToSpeechClient 인스턴스
    """
    cred_path = os.getenv("GOOGLE_APPLICATION_CREDENTIALS")
    if cred_path and os.path.exists(cred_path):
        # 서비스 계정 키 파일을 직접 로딩해서 명시적으로 인증
        creds = service_account.Credentials.from_service_account_file(cred_path)
        return texttospeech.TextToSpeechClient(credentials=creds)
    # 환경에 이미 ADC가 구성되어 있다면 이를 사용
    return texttospeech.TextToSpeechClient()  # fallback (ADC)

In [ ]:
# 목적:
# - 순수 텍스트를 Google Cloud TTS로 합성해 16-bit PCM WAV(모노, 24kHz) 파일로 저장한다.

def gcp_tts_synthesize(
    text: str,
    out_wav: str = "gcp_out.wav",
    language_code: str = "ko-KR",
    voice_name: str | None = None,
    speaking_rate: float = 1.0
):
    """
    텍스트를 GCP TTS로 합성하여 16-bit PCM WAV 파일로 저장합니다.

    파라미터:
      - text: 합성할 순수 텍스트 (SSML 아님)
      - out_wav: 출력 WAV 파일 경로
      - language_code: 언어 코드 (예: 'ko-KR', 'en-US' 등)
      - voice_name: 구체적 보이스 지정(선택). 지정하지 않으면 언어코드에 맞는 기본 보이스.
      - speaking_rate: 발화 속도(배율). 1.0=기본, 0.8=느리게, 1.2=빠르게 등

    동작:
      - AudioEncoding.LINEAR16 (16-bit PCM) + 24kHz 샘플레이트로 합성
      - wave 모듈로 1채널/16-bit/24kHz WAV 헤더를 기록한 뒤 바이트 스트림을 저장

    반환:
      - 생성된 WAV 파일 경로(out_wav)
    """
    client = make_tts_client()

    # 보이스 선택 파라미터 구성
    voice_params = {"language_code": language_code}
    if voice_name:
        voice_params["name"] = voice_name
    voice = texttospeech.VoiceSelectionParams(**voice_params)

    # 오디오 출력 설정:
    # - LINEAR16: 16-bit PCM (WAV에 적합)
    # - sample_rate_hertz: 24000 (아래 wave 헤더와 동일하게 설정)
    # - speaking_rate: 말하기 속도 제어(1.0=기본)
    audio_config = texttospeech.AudioConfig(
        audio_encoding=texttospeech.AudioEncoding.LINEAR16,
        sample_rate_hertz=24000,
        speaking_rate=speaking_rate,
    )

    # 텍스트 입력
    resp = client.synthesize_speech(
        input=texttospeech.SynthesisInput(text=text),
        voice=voice,
        audio_config=audio_config,
    )

    # WAV 파일로 저장
    # - 채널: 1(모노)
    # - 샘플폭: 2바이트(16-bit PCM)
    # - 샘플레이트: 24000 Hz
    with wave.open(out_wav, "wb") as f:
        f.setnchannels(1)
        f.setsampwidth(2)
        f.setframerate(24000)
        f.writeframes(resp.audio_content)

    return out_wav

In [ ]:
# 목적:
# - GCP 서비스 계정 키(.json) 경로와 최소 메타정보를 점검하고
# - 실제 합성 요청 한 번으로 TTS 연결(인증/권한/네트워크 등)을 스모크 테스트합니다.
# - 성공하면 ./data/speech/bench_gcp.wav 파일이 생성됩니다.

# 인증/환경 점검 스니펫
import os, json

cred = os.getenv("GOOGLE_APPLICATION_CREDENTIALS")
if cred and os.path.exists(cred):
    # 키 파일이 존재하면 최소 메타 정보만 확인 (민감 정보 출력 금지)
    with open(cred, "r", encoding="utf-8") as f:
        meta = json.load(f)
    print("service_account:", meta.get("client_email", "-"))
else:
    # 키 경로가 비어 있거나 파일이 없을 때 안내
    print("경고: 서비스 계정 키 경로를 설정하세요. (환경변수 GOOGLE_APPLICATION_CREDENTIALS)")

# 샘플 합성 전 간단 요청 한 번으로 연결 확인
# - gcp_tts_synthesize 는 앞서 정의한 최소 합성 함수입니다.
try:
    _ = gcp_tts_synthesize(
        "안녕하세요. 구글 클라우드 TTS 데모입니다.",
        out_wav="./data/speech/bench_gcp.wav",
        language_code="ko-KR"
    )
    print("TTS OK → bench_gcp.wav")
except Exception as e:
    print("TTS 실패:", e)



### 2. 텍스트 정규화/문장 분할 실습(소수점 보호)
- 예시 문장: `오늘 기온은 23.5도, 바람이 강합니다! 1kg짜리 샘플과 A/S 절차를 안내합니다.`
- 목표: 숫자/단위/기호를 정규화하고, 소수점은 분할 대상에서 제외하여 자연스러운 억양을 얻습니다.
- 결과: 전체 1샷 합성(`tts_full.wav`)과 문장 단위 합성(`tts_sent_*.wav`) 비교 청취.


In [ ]:
EX_TEXT = "오늘 기온은 23.5도, 바람이 강합니다! 1kg짜리 샘플과 A/S 절차를 안내합니다."

In [ ]:
# 소수점 보호 정규화: 23.5 같은 패턴은 유지
# - 이 함수는 소수점 자체를 건드리지 않습니다(= 23.5 → 그대로 유지).
# - 특정 표기만 발화 친화적으로 바꿉니다: "A/S, AS" → "에이에스", "1kg" → "1 킬로그램",
#   그리고 과도한 공백을 1칸으로 정리합니다.
def normalize_keep_decimal(text: str) -> str:
    """
    특정 약어/단위 표기를 TTS 친화적으로 정규화합니다.
    소수점 숫자(예: 23.5)는 변경하지 않습니다.

    Args:
        text (str): 원문 텍스트

    Returns:
        str: 정규화된 텍스트
    """
    txt = text

    # [A/S → 에이에스]
    # - 대소문자 무시(re.IGNORECASE)
    # - 단어 경계(\b)로 'A/S'가 단독 토큰일 때만 치환
    txt = re.sub(r"\bA/S\b", "에이에스", txt, flags=re.IGNORECASE)

    # [AS → 에이에스]
    # - 'AS' 단독 토큰을 '에이에스'로 치환 (예: 'AS 규정')
    # - 'ASAP' 같은 단어 내부는 \b 경계 때문에 치환되지 않음
    txt = re.sub(r"\bAS\b", "에이에스", txt, flags=re.IGNORECASE)

    # [1kg → 1 킬로그램]
    # - 정수 뒤에 붙은 'kg' 단위를 한글로 풀어 읽도록 변경
    # - 예: '1kg' → '1 킬로그램', '2 kg' → '2 킬로그램'
    # - 참고: 소수(예: 1.5kg)는 현재 패턴(\d+)상 매칭되지 않음
    txt = re.sub(r"(\d+)\s*kg\b", r"\1 킬로그램", txt, flags=re.IGNORECASE)

    # [공백 정리]
    # - 연속 공백을 1칸으로, 양끝 공백 제거
    txt = re.sub(r"\s+", " ", txt).strip()

    return txt

In [ ]:
# 소수점 제외 문장 분할: 숫자 소수점은 그대로 두고, .?!; 만 문장 경계 처리
# - 핵심 아이디어:
#   (1) 먼저 숫자 사이의 마침표(소수점)만 임시 토큰으로 치환하여 "문장 구분 마침표"와 구분
#   (2) ., !, ?, ; 기준으로 split 하되 구분기호를 보존하여 원문 느낌 유지
#   (3) 분할 후 임시 토큰을 다시 '.'로 복원
# - 장점: "23.5도" 같은 소수점이 중간에서 끊기지 않음
def split_sentences_keep_decimal(text: str):
    """
    텍스트를 문장 단위로 분할하되, '숫자.숫자' 형태의 소수점은 보존합니다.

    Args:
        text (str): 원본 텍스트

    Returns:
        List[str]: 문장 단위로 분할된 문자열 리스트 (구분기호 포함)
    """
    # 1) '숫자.숫자' 패턴 보호: 소수점만 임시 토큰으로 치환
    #    예: "23.5" → "23<DOT>5"
    placeholder = "<DOT>"
    protected = re.sub(r"(\d)\.(\d)", rf"\1{placeholder}\2", text)

    # 2) 문장 구분 기호 단위로 분할하되, 구분기호를 그룹으로 캡처하여 보존
    #    - 예: "안녕. 너 누구야?" → ["안녕", ".", " 너 누구야", "?"]
    parts = re.split(r"([.!?;])", protected)

    # 3) 토막난 조각을 다시 문장 단위로 재조합 (구분기호 포함)
    sents, buf = [], ""
    for p in parts:
        if re.match(r"[.!?;]", p):
            # 구분기호를 현재 버퍼 끝에 붙이고, 하나의 문장으로 확정
            buf += p
            sents.append(buf.strip())
            buf = ""
        else:
            # 일반 텍스트는 버퍼에 누적
            buf += p
    if buf.strip():
        # 마지막에 구분기호 없이 남은 잔여 텍스트가 있으면 문장으로 추가
        sents.append(buf.strip())

    # 4) 보호 토큰을 소수점으로 복원
    sents = [s.replace(placeholder, ".") for s in sents]

    # 5) 공백/빈 문자열 정리 후 반환
    #    (쉼표 뒤 휴지는 SSML에서 <break>로 제어하는 편이 명시적)
    return [s for s in sents if s]

In [ ]:
# 목적: 정규화/문장분할 유틸을 적용한 뒤, 전체 1샷 합성과 문장 단위 합성을 각각 수행

# 1) 정규화 + 문장 분할
norm = normalize_keep_decimal(EX_TEXT)          # 예: "A/S" → "에이에스", "1kg" → "1 킬로그램" 등 (소수는 그대로)
sents = split_sentences_keep_decimal(norm)      # 소수점("23.5")을 보호한 채 .?!; 기준으로 문장 분리

print("정규화:", norm)
print("문장 분할:", sents)                      # 디버깅/검수용: 실제로 어떻게 쪼개졌는지 확인

# 2) 전체 1샷 합성 (문장 전체를 한 번에 합성)
#    - 간단하고 매끄럽게 들릴 가능성이 높음(문장 경계 이어짐 자연스러움)
#    - 다만 특정 구간만 속도/톤을 바꾸는 정밀 제어는 어려움
full_out = "./data/speech/tts_full.wav"
gcp_tts_synthesize(
    norm,
    out_wav=full_out,
    language_code="ko-KR",      # 필요 시 보이스 지정: voice_name="ko-KR-Standard-A" 등
    # speaking_rate=1.0,
)
print("saved:", full_out)

# 3) 문장 단위 합성 (예시로 처음 2문장만)
#    - 각 문장을 개별 파일로 저장 → UI/앱에서 재사용/재배열/재생 제어에 유리
#    - 문장 사이 공백/휴지는 SSML(<break>)로 처리하는 것이 더 명확
for i, s in enumerate(sents[:2], start=1):
    out = f"./data/speech/tts_sent_{i}.wav"
    gcp_tts_synthesize(
        s,
        out_wav=out,
        language_code="ko-KR",
        # speaking_rate=1.0,
        # voice_name="ko-KR-Standard-A",
    )
    print("saved:", out)


### 3. 보이스/언어 탐색 및 번역 (멀티스피커·멀티언어 확인)
- Google Cloud TTS 보이스 목록을 조회하여 언어/성별/샘플레이트를 빠르게 확인합니다.
- 멀티언어로 같은 문장을 합성해 차이를 청취합니다.


In [ ]:
# 목적: Google Cloud TTS에서 사용 가능한 보이스 목록을 조회하고, 특정 언어 코드(lang)에 해당하는 보이스만 화면에 요약 출력

def list_voices(lang: str = "ko-KR", limit: int = 20):
    """
    파라미터:
      - lang: 필터링할 언어 코드 (예: "ko-KR", "en-US", "ja-JP")
      - limit: 출력할 최대 개수 (조회된 보이스가 매우 많을 수 있으므로 화면 출력 수를 제한)

    동작 개요:
      1) make_tts_client()로 인증된 TTS 클라이언트를 생성
      2) client.list_voices()로 전체 보이스 메타를 가져옴
      3) language_codes에 lang이 포함된 항목만 필터링
      4) 보이스 이름, 성별(SSML 기준), 지원 언어 코드, 샘플레이트(Hz)를 출력
    """
    client = make_tts_client()  # 앞에서 정의한 인증 유틸 사용(서비스 계정/ADC)

    # 전체 보이스 중에서 lang을 지원하는 보이스만 선별
    voices = [v for v in client.list_voices().voices if lang in v.language_codes]

    print(f"[{lang}] {len(voices)} voices")
    # 너무 많은 보이스가 나올 수 있으므로 limit까지만 출력
    for v in voices[:limit]:
        # 안전하게 속성 접근(일부 필드가 비어 있을 수 있음)
        name = getattr(v, "name", "-")
        gender = getattr(v, "ssml_gender", None)
        gender_name = getattr(gender, "name", "-") if gender is not None else "-"
        langs = getattr(v, "language_codes", ["-"])
        hz = getattr(v, "natural_sample_rate_hertz", 0) or 0

        print(name, gender_name, langs, f"{hz}Hz")

# 사용 예시
list_voices("ko-KR")   # 한국어 보이스 조회
# list_voices("en-US") # 영어 보이스 조회
# list_voices("ja-JP") # 일본어 보이스 조회



In [ ]:
# 멀티언어 합성 비교(같은 문장)
langs = ["ko-KR", "en-US", "ja-JP"]
for lc in langs:
    out = f"./data/speech/gcp_{lc}.wav"
    gcp_tts_synthesize("안녕하세요. TTS 비교입니다.", out_wav=out, language_code=lc)
    print("saved:", out)


In [ ]:
# 목적:
# - "원문 텍스트"를 각 언어로 번역(MT)한 뒤, 번역 결과를 그 언어 보이스로 TTS 합성까지 자동 수행
# - 번역 텍스트도 함께 출력/저장하여 비교 가능

# 번역 클라이언트 생성
def make_translate_client():
    cred_path = os.getenv("GOOGLE_APPLICATION_CREDENTIALS")
    if cred_path and os.path.exists(cred_path):
        creds = service_account.Credentials.from_service_account_file(cred_path)
        return translate.Client(credentials=creds)
    return translate.Client()  # ADC fallback

In [ ]:
# 간단 번역 함수

import html

def translate_text(text: str, target_lang: str, source_lang: str | None = None) -> str:
    """
    target_lang: 'en', 'ja', 'ko' 같은 2글자 또는 BCP-47 코드 (예: 'en', 'ja', 'ko')
    source_lang: 원문 언어(모르면 None → 자동감지)
    """
    client = make_translate_client()
    resp = client.translate(text, target_language=target_lang, source_language=source_lang)
    # v2 API는 HTML-escaped 문자열을 반환하므로 unescape, 이상한 발음/기호 낭독 방지
    return html.unescape(resp["translatedText"])


In [ ]:
# TTS 언어코드(voice) ↔ 번역 언어코드 매핑
# - TTS는 'ko-KR', 'en-US', 'ja-JP' 등 로케일 코드
# - 번역은 보통 'ko', 'en', 'ja' 등 간단 코드 사용
TTS_TO_TRANS = {
    "ko-KR": "ko",
    "en-US": "en",
    "ja-JP": "ja",
    # 필요 시 추가: "zh-CN":"zh-CN", "de-DE":"de", ...
}

In [ ]:
VOICE_MAP = {
    "ko-KR": "ko-KR-Neural2-A",
    "en-US": "en-US-Neural2-D",
    "ja-JP": "ja-JP-Neural2-B",
}

In [ ]:
# 목적:
# - 입력 문장을 타깃 언어별로 번역한 뒤 해당 언어 보이스로 TTS(WAV) 생성
# - 원문(ko-KR)도 함께 합성해 비교용 파일 저장
# - 생성 파일/텍스트/언어 정보를 JSON 메타로 기록

def translate_and_tts(
    text_ko: str,
    targets_tts: list[str] = ["ko-KR", "en-US", "ja-JP"],
    out_dir: str = "./data/speech",
    base_name: str = "mt_tts"
):
    """
    text_ko: 원문(여기서는 한국어 예시). 어떤 언어여도 source_lang=None이면 자동감지
    targets_tts: TTS 언어코드 목록 (각각 번역 후 해당 언어 보이스로 합성)
    out_dir: 오디오/결과 저장 폴더
    base_name: 출력 파일 접두어
    """
    os.makedirs(out_dir, exist_ok=True)
    results = []

    # 원문도 그대로 TTS(비교용)
    out_src = os.path.join(out_dir, f"{base_name}_src_ko-KR.wav")
    gcp_tts_synthesize(text_ko, out_wav=out_src, language_code="ko-KR", voice_name=VOICE_MAP.get("ko-KR"))
    print("saved:", out_src)
    results.append({"lang_tts":"ko-KR", "text": text_ko, "file": out_src, "type":"source"})

    # 타겟별 번역 → TTS
    for lc in targets_tts:
        # 1) 번역 언어코드 결정
        trg = TTS_TO_TRANS.get(lc)
        if not trg:
            print(f"[warn] 번역코드 매핑이 없습니다: {lc} → 건너뜀")
            continue

        # 2) 번역 수행 (source_lang=None: 자동감지)
        mt = translate_text(text_ko, target_lang=trg, source_lang=None)

        # 3) 해당 언어 보이스로 TTS
        out = os.path.join(out_dir, f"{base_name}_{lc}.wav")
        gcp_tts_synthesize(mt, out_wav=out, language_code=lc, voice_name=VOICE_MAP.get(lc))
        print(f"saved: {out}  |  text({lc}/{trg}): {mt}")

        results.append({"lang_tts": lc, "lang_mt": trg, "text": mt, "file": out, "type":"translated"})

    # 번역/합성 결과 텍스트도 JSON으로 보관
    meta_path = os.path.join(out_dir, f"{base_name}_results.json")
    with open(meta_path, "w", encoding="utf-8") as f:
        json.dump(results, f, ensure_ascii=False, indent=2)
    print("meta saved:", meta_path)

    return results


In [ ]:

# 동일 문장을 ko/en/ja로 번역 후 해당 언어 보이스로 읽기
_ = translate_and_tts(
    text_ko="안녕하세요. TTS 비교를 위해 번역된 음성도 함께 들어봅니다.",
    targets_tts=["ko-KR", "en-US", "ja-JP"],
    out_dir="./data/speech",
    base_name="demo_mt_tts"
)

### 4. 보이스 선택 가이드 (GCP)
- `list_voices(lang)`로 언어별 보이스를 조회하고, HD/Chirp3 계열을 우선 검토합니다.
- `ssml_gender`, `natural_sample_rate_hertz`를 확인해 품질/취향을 맞춥니다.



In [ ]:
# 목적:
# - Google Cloud TTS의 보이스 목록을 조회한 뒤,
#   간단한 휴리스틱(이름에 'Chirp3-HD' 포함 우선 → natural_sample_rate_hertz 높은 순)으로
#   상위 N개를 정렬/출력하고, 그 보이스명 리스트를 반환합니다.
# - 이어서 상위 2개 보이스로 샘플 WAV를 생성합니다.


import os
from google.cloud import texttospeech

def list_voices_ranked(lang: str = "ko-KR", limit: int = 10):
    """
    언어 코드(lang)를 지원하는 보이스들만 모아 간단 점수로 정렬한 뒤,
    상위 limit개를 콘솔에 출력하고 보이스명 리스트를 반환합니다.

    정렬 점수(score):
      - 'Chirp3-HD'가 이름에 포함되어 있으면 hd=1, 아니면 0
      - 같은 hd 그룹 내에서 natural_sample_rate_hertz(샘플레이트)가 큰 순으로 정렬
    """
    client = make_tts_client()
    # 해당 언어를 지원하는 보이스만 필터링
    voices = [v for v in client.list_voices().voices if lang in v.language_codes]

    # 간단 가중치: HD(이름에 'Chirp3-HD') 우선 → 샘플레이트 우선
    def score(v):
        name = getattr(v, "name", "") or ""
        hd = 1 if "Chirp3-HD" in name else 0
        hz = getattr(v, "natural_sample_rate_hertz", 0) or 0
        return (hd, hz)

    voices.sort(key=score, reverse=True)

    # 결과 요약 출력
    print(f"[{lang}] {len(voices)} voices (top {min(limit, len(voices))})")
    for v in voices[:limit]:
        name = getattr(v, "name", "-")
        gender = getattr(v, "ssml_gender", None)
        gender_name = getattr(gender, "name", "-") if gender is not None else "-"
        langs = getattr(v, "language_codes", ["-"])
        hz = getattr(v, "natural_sample_rate_hertz", 0) or 0
        print(name, gender_name, langs, f"{hz}Hz")

    # 상위 보이스명만 반환(후속 합성에서 voice_name으로 사용)
    return [getattr(v, "name", "") for v in voices[:limit]]

# ── 예시 실행 및 2개 샘플 생성 ─────────────────────────────────────────
best = list_voices_ranked("ko-KR", limit=5)

# 출력 폴더 보장
os.makedirs("./data/speech", exist_ok=True)

for name in best[:2]:
    out = f"./data/speech/gcp_{name}.wav"  # 보이스명을 파일명에 포함(재현성 확인에 유리)
    gcp_tts_synthesize(
        "보이스 선택 가이드 샘플 문장입니다.",
        out_wav=out,
        language_code="ko-KR",
        voice_name=name
    )
    print("saved:", out)


### 5. SSML 미세조정 체크리스트 (A/B)
- **rate**: 100% → 90% → 80%
- **pitch**: 0st → +2st → -2st
- **break**: 0ms → 200ms → 400ms
- 문장을 고정하고 위 파라미터를 한 번에 한 개씩 바꿔 A/B 비교합니다.

In [ ]:
# 재현성 확보용: 고정 보이스/기본 문장
VOICE_NAME = "ko-KR-Neural2-A"   
BASE_SENT = "오늘 일정과 공지 사항을 안내드립니다."

# ── SSML 합성 함수(먼저 정의) ─────────────────────────────────
from google.cloud import texttospeech

def gcp_tts_ssml(ssml: str, out_wav: str, language_code: str = "ko-KR", voice_name: str | None = None):
    """
    목적:
      - SSML 문자열을 Google Cloud TTS로 합성하여 16-bit PCM(WAV) 파일로 저장합니다.
      - SSML(rate, pitch, break, say-as, emphasis 등)을 해석하려면 반드시 SynthesisInput(ssml=...)를 사용해야 합니다.

    파라미터:
      - ssml: 합성할 SSML 문자열 (예: "<speak>...<prosody rate='90%'>...</prosody>...</speak>")
      - out_wav: 출력 WAV 경로 (예: "./data/speech/out.wav")
      - language_code: 보이스 언어 코드 (예: "ko-KR", "en-US")
      - voice_name: 특정 보이스명(선택). 지정하지 않으면 언어 코드에 맞는 기본 보이스가 선택될 수 있음.
    """
    # 인증 클라이언트 생성(서비스 계정/ADC)
    client = make_tts_client()

    # 보이스 선택: 언어 코드와(선택) 특정 보이스명을 함께 전달
    voice = texttospeech.VoiceSelectionParams(
        language_code=language_code,
        name=voice_name
    )

    # 오디오 설정: LINEAR16(16-bit PCM), 24kHz
    audio_config = texttospeech.AudioConfig(
        audio_encoding=texttospeech.AudioEncoding.LINEAR16,
        sample_rate_hertz=24000,
    )

    # SSML 입력으로 합성 요청 (중요: text=가 아닌 ssml= 사용)
    resp = client.synthesize_speech(
        input=texttospeech.SynthesisInput(ssml=ssml),
        voice=voice,
        audio_config=audio_config
    )

    # 파일 저장 (out_wav 활용)
    os.makedirs(os.path.dirname(out_wav) or ".", exist_ok=True)
    with open(out_wav, "wb") as f:
        f.write(resp.audio_content)

In [ ]:
# ── SSML 미세조정 체크리스트 (A/B) ────────────────────────────
# 목적: 같은 문장을 유지한 채 'rate(속도)'만 바꿔가며 청취 A/B 비교를 수행합니다.

RATES   = [100, 90, 80]        # 발화 속도(%) — 100=기본, 90=10% 느리게, 80=20% 느리게
PITCHES = ["0st", "+2st", "-2st"]   # (후속 실험용) 피치 변화: 반음 단위(semitone)
BREAKS  = ["0ms", "200ms", "400ms"] # (후속 실험용) 휴지 길이: 밀리초

# 1) rate만 바꾸는 A/B
for r in RATES:
    # SSML 구성: prosody의 rate에 퍼센트 단위로 속도 지정
    # - BASE_SENT는 동일하게 유지하여 속도 변화만 체감하도록 함
    ssml = f"""
<speak>
  <p><prosody rate="{r}%">{BASE_SENT}</prosody></p>
</speak>
""".strip()

    # 출력 파일명: 설정값(r)을 포함해 비교 결과 정리/추적이 쉽도록 함
    out = f"./data/speech/gcp_ab_rate_{r}.wav"

    # 합성 실행:
    # - language_code: "ko-KR" 고정(보이스/언어 일정)
    # - voice_name: VOICE_NAME을 지정해 실험 재현성 확보(미지정 시 기본 보이스가 바뀔 수 있음)
    gcp_tts_ssml(ssml, out, language_code="ko-KR", voice_name=VOICE_NAME)

    # 저장 경로 출력
    print("saved:", out)

In [ ]:
# 2) pitch만 바꾸는 A/B
# 목적:
# - 같은 문장(BASE_SENT)을 유지하고 prosody의 pitch 값만 변경하여 억양(피치) 차이를 청취 비교합니다.
# 포인트:
# - pitch 단위 'st'는 semitone(반음)입니다. "+2st"는 반음 2단계 ↑, "-2st"는 2단계 ↓.
# - 보이스/언어에 따라 pitch 변화가 작게 들리거나 제한될 수 있습니다(일부 엔진은 범위를 캡/무시).
# - rate(속도)와 동시 변경 시 비교가 흐려지므로 여기서는 pitch만 바꿉니다.
# - 파일명에 '+'를 직접 쓰면 플랫폼/도구에 따라 취급이 애매할 수 있어 safe_p로 대체합니다.

for p in PITCHES:
    safe_p = p.replace('+','plus_')  # 파일명 안전 처리: "+2st" → "plus_2st"

    # SSML 구성: prosody의 pitch만 변경
    # - 예) "0st"(기본), "+2st"(조금 높게), "-2st"(조금 낮게)
    # - 지나치게 큰 값은 인위적/부자연스러울 수 있으니 ±1~±4st 범위에서 먼저 비교 권장
    ssml = f"""
<speak>
  <p><prosody pitch="{p}">{BASE_SENT}</prosody></p>
</speak>
""".strip()

    # 출력 파일명: 어떤 설정으로 합성했는지 추적 용이하도록 pitch 값을 반영
    out = f"./data/speech/gcp_ab_pitch_{safe_p}.wav"

    # 합성 실행:
    # - language_code: "ko-KR"으로 고정(언어/보이스 일관성 유지)
    # - voice_name: VOICE_NAME 고정으로 재현성 확보(미지정 시 기본 보이스가 바뀔 수 있음)
    gcp_tts_ssml(ssml, out, language_code="ko-KR", voice_name=VOICE_NAME)

    # 저장 경로 출력
    print("saved:", out)



In [ ]:
# 3) break만 바꾸는 A/B
# 목적:
# - 같은 문장을 유지하고 <break> 시간만 바꿔, 구두적 '쉼' 길이의 체감 차이를 청취 비교합니다.
# 설계 포인트:
# - "0ms"일 땐 아예 <break>를 넣지 않아 엔진의 기본 휴지만 두도록 함.
# - 그 외에는 "오늘 일정과 | 공지 사항…" 사이에 <break time="..."/>를 삽입.
# - 한국어 문장 경계에서는 보통 150–300ms가 자연스럽고, 항목 나열/전환 강조엔 300–500ms도 사용합니다.
# 주의:
# - 일부 보이스/엔진은 너무 긴/짧은 break를 클램핑(무시/제한)할 수 있습니다.
# - break 남발은 '기계적' 느낌을 줄 수 있으니 문장 경계나 의미 단락에만 배치하세요.

for b in BREAKS:
    if b == "0ms":
        # 휴지 없음: 엔진의 기본 prosody에만 의존
        ssml = f"""
<speak>
  <p>{BASE_SENT}</p>
</speak>
""".strip()
    else:
        # 의미 단락 사이에 명시적 휴지 삽입
        ssml = f"""
<speak>
  <p>오늘 일정과 <break time="{b}"/> 공지 사항을 안내드립니다.</p>
</speak>
""".strip()

    # 파일명에 break 길이를 반영해 비교/정리 용이
    out = f"./data/speech/gcp_ab_break_{b}.wav"

    # 합성 실행 (언어/보이스 고정으로 재현성 확보)
    gcp_tts_ssml(ssml, out, language_code="ko-KR", voice_name=VOICE_NAME)
    print("saved:", out)



In [ ]:
# ── 최종 데모: prosody / break / say-as / emphasis ─────────────────────────────
# 목적:
# - 한 SSML 스니펫 안에서 여러 제어 태그를 종합적으로 사용해 TTS 스타일을 시연합니다.
#   * <prosody rate/pitch> : 발화 속도/피치 조정
#   * <break>              : 의미 단락 사이에 휴지(일시정지) 삽입
#   * <emphasis>           : 단어 강조(보이스/언어에 따라 효과 차이)
#   * <say-as>             : 철자(문자 단위)나 숫자 읽기 방식 지정
#

ssml = """
<speak>
  <!-- 기본 문장: 비교 기준 -->
  <p>기본 문장입니다.</p>

  <!-- prosody: 속도(rate=85%), 피치(pitch=+2st) -->
  <p><prosody rate="85%" pitch="+2st">속도를 줄이고 피치를 올린 문장입니다.</prosody></p>

  <!-- break: 쉼표 뒤에 400ms 휴지로 리듬 강조 -->
  <p>쉼표 뒤에 <break time="400ms"/> 짧은 휴지를 넣었습니다.</p>

  <!-- emphasis: 단어 강조(보이스에 따라 강도/톤 차이) -->
  <p><emphasis level="moderate">강조된 단어</emphasis>를 포함했습니다.</p>

  <!-- say-as: 읽기 방식 지정
       - characters: 철자대로 읽기 (예: A/S → 에이 슬래시 에스)
       - cardinal  : 기수(숫자)로 읽기 (예: 1234 → 천이백삼십사)
       * 실제 발음은 언어/보이스 따라 다를 수 있습니다. -->
  <p>숫자 읽기: <say-as interpret-as="characters">A/S</say-as>,
               <say-as interpret-as="cardinal">1234</say-as></p>
</speak>
""".strip()

# 합성 실행: SSML은 반드시 SynthesisInput(ssml=...)로 전달해야 태그가 반영됩니다.
gcp_tts_ssml(
    ssml,
    "./data/speech/gcp_ssml.wav",
    language_code="ko-KR",
    voice_name=VOICE_NAME  # 예: "ko-KR-Chirp3-HD-Callirrhoe"
)

print("saved: ./data/speech/gcp_ssml.wav")